### **Package**

In [1]:
import subprocess
import sys
import os

# Stop showing warning tips below
# <frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
# <frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
# <frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
# <frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
# <frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
# <frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
# <frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
sys.stderr = open(os.devnull, 'w')

print(sys.version)

try:
    import pythermalcomfort
    print("pythermalcomfort is already installed.")
except ImportError:
    print("pythermalcomfort is not installed. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pythermalcomfort"])
    print("pythermalcomfort installed.")

try:
    from autogluon.tabular import TabularPredictor
    print("AutoGluon is already installed.")
except ImportError:
    print("AutoGluon is not installed. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "autogluon.tabular"])
    print("AutoGluon installed.")

try:
    import pymoo
    print("pymoo is already installed.")
except ImportError:
    print("pymoo is not installed. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pymoo"])
print("pymoo installed.")

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
pythermalcomfort is not installed. Installing...
pythermalcomfort installed.
AutoGluon is not installed. Installing...
AutoGluon installed.
pymoo is not installed. Installing...
pymoo installed.


In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

from pythermalcomfort.models import pmv
from pythermalcomfort.utilities import v_relative, clo_dynamic
from autogluon.tabular import TabularPredictor

from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.visualization.scatter import Scatter
from pymoo.core.mutation import Mutation
from pymoo.operators.crossover.sbx import SBX

from pymoo.algorithms.moo.moead import MOEAD
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.core.population import Population, Individual
from pymoo.core.sampling import Sampling

from pymoo.indicators.hv import HV

import time

from scipy.stats import entropy


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Class: objectives f1 f2**

In [3]:
class f1_cost:
    def __init__(self, predictor):

        """
        predictor: Autogluon predictor
        """
        self.predictor = predictor
        self.columns = ['occupant_count [number]',
                        'air_temperature [Celsius]',
                        'indoor_relative_humidity [%]',
                        'dry_bulb_temp [Celsius]',
                        'outdoor_relative_humidity [%]',
                        'wind_speed [m/s]',
                        'global_horizontal_solar_radiation [W/m2]']

    def calculate_cost(self, features_batch, room_area_list):
        """
        In features_batch:
        occ: Occupant count (number of people)
        indoor_t: Indoor air temperature (°C)
        indoor_rh: Indoor relative humidity (%)
        outdoor_t: Outdoor dry bulb temperature (°C)
        outdoor_rh: Outdoor relative humidity (%)
        wind_speed: Outdoor wind speed (m/s)
        solar_radiation: Global horizontal solar radiation (W/m²)
        predictor: Autogluon predictor
        """

        data = pd.DataFrame(features_batch, columns=self.columns)
        predctions = self.predictor.predict(data, model='WeightedEnsemble_L2')

        results = sum(predctions * (room_area_list/sum(room_area_list)))
        return results



class f2_pmv:
    def __init__(self):
      pass

    def calculate_pmv(self, features_batch, room_occ_list):
        """
        indoor_t: Air temperature (°C)
        tr: Mean radiant temperature (°C)
        v: Air velocity (m/s)
        indoor_rh: Relative humidity (%)
        met: Metabolic rate (met)
        clo: Clothing insulation (clo)
        """
        pmv_results = np.array([])

        for feature in features_batch:
            indoor_t, tr, v, indoor_rh, met, clo = feature
            v_r = v_relative(v=v, met=met)
            clo_d = clo_dynamic(clo=clo, met=met)

            result = pmv(tdb=indoor_t, tr=tr, vr=v_r, rh=indoor_rh, met=met, clo=clo)
            pmv_results = np.append(pmv_results,result)

        results = sum(abs(pmv_results) * room_occ_list) / sum(room_occ_list)
        return results

###**Class: plot_2D_display**

In [4]:
class plot_2D_display:
    def __init__(self, num_variables, initial_position, room_types, space_width):
        """
        num_variables (int): number of decision variables
        initial_position (np.array): initial position of walls
        room_types (list): room types
        """

        self.initial_position = initial_position
        self.num_variables = num_variables
        self.room_types = room_types
        self.space_width = space_width
        self.y_height = space_width * 3  # Fixed the height for rooms to plot

        # Define colors for different room types
        self.room_colors = {
            'office': '#c8dbf3',
            'meeting_room': '#d5e8d4',
            'closed': '#f8cecc'
            }

    def plot(self):

        # Y values
        y_values_0 = np.zeros_like(self.initial_position)
        y_values_1 = np.full_like(self.initial_position, self.y_height)

        # # Plot points, lines and room details
        plt.figure(figsize=(8, 5))
        self._plot_points_and_lines(y_values_0, y_values_1)
        self._plot_room_details(y_values_0, y_values_1)

        # Configure the plot's axes, labels, patches
        plt.axis('equal')
        plt.xlabel("X")
        plt.gca().axes.get_yaxis().set_visible(False)
        plt.title("2D Room Plots")
        office_patch = mpatches.Patch(color= self.room_colors.get('office'),
                                      label='Office')
        meeting_room_patch = mpatches.Patch(color= self.room_colors.get('meeting_room'),
                                            label='Meeting room')
        closed_room_patch = mpatches.Patch(color= self.room_colors.get('closed'),
                                            label='Closed room')
        room_length_line = mlines.Line2D([], [], color='black', label=f'Room length: {r"$x_i$"}')
        room_width_line = mlines.Line2D([], [], color='black', label=f'Room width: {self.space_width}')
        plt.legend(handles=[office_patch, meeting_room_patch, closed_room_patch,
                            room_length_line, room_width_line],
                   loc='upper left')
        plt.show()

    def _plot_points_and_lines(self, y_values_0, y_values_1):

        # points and lines
        plt.scatter(self.initial_position, y_values_0, color='black', s=1)
        plt.scatter(self.initial_position, y_values_1, color='black', s=1)
        plt.plot(self.initial_position, y_values_0, color='black', linestyle='-', linewidth=1)
        plt.plot(self.initial_position, y_values_1, color='black', linestyle='-', linewidth=1)
        for x, y0, y1 in zip(self.initial_position, y_values_0, y_values_1):
            plt.plot([x, x], [y0, y1], color='black', linestyle='-', linewidth=1)

    def _plot_room_details(self, y_values_0, y_values_1):
        j = 0
        for i in range(len(self.initial_position) - 1):
            x0 = self.initial_position[i]
            x1 = self.initial_position[i + 1]

            if x0 == x1:
                continue

            else:

              # Room number
              index_x = x0 + 0.5
              plt.text(index_x, self.y_height * 0.8, str(j+1), fontsize=9, ha='center', color='black')

              # Room area
              room_length = x1 - x0
              room_width = self.space_width
              height = y_values_1[i] - y_values_0[i]
              area = round(room_length * room_width, 1)
              mid_x = (x0 + x1) / 2
              plt.text(mid_x, self.y_height * 0.1, f"{area} m²", fontsize=9, ha='center', color='black')

              # Color
              room_type = self.room_types[j]
              room_color = self.room_colors.get(room_type, 'lightgray')
              plt.fill_between([x0, x1], y_values_0[i], y_values_1[i], color=room_color, alpha=1)
              j += 1

### **Class: flexible_wall_problem**

In [5]:
class flexible_wall_problem(Problem):
#class flexible_wall_problem:
    def __init__(self, num_decision_variables, initial_position, fixed_wall_list,
                 room_types, required_num_meeting_room, space_width, room_area_min,
                 initial_occ_list, total_occ, occ_min_area,
                 th_zone_list,
                 predictor,
                 indoor_t, indoor_rh, outdoor_t, outdoor_rh, wind_speed, solar_radiation,
                 clo, met, v):
        """
        num_variables (int): number of decision variables
        initial_position (np.array): initial position of walls
        room_types (list): room types
        """
        # Check num_input
        if len(initial_position[0]) != num_decision_variables:
            raise ValueError(f"len(initial_position) {len(initial_position)} does not match "
                             f"num_decision_variables {num_decision_variables}.")

        if len(fixed_wall_list) != num_decision_variables:
            raise ValueError(f"len(fixed_wall_list) {len(fixed_wall_list)} does not match "
                             f"num_decision_variables {num_decision_variables}.")

        self.num_decision_variables = num_decision_variables
        self.position = initial_position
        self.room_types = room_types
        self.initial_occ_list = initial_occ_list
        self.total_occ = total_occ
        self.occ_min_area = occ_min_area
        self.required_num_meeting_room = required_num_meeting_room
        self.space_width = space_width
        self.room_area_min = room_area_min

        # Storing additional parameters as class attributes
        self.predictor = predictor
        self.indoor_t = indoor_t
        self.indoor_rh = indoor_rh
        self.outdoor_t = outdoor_t
        self.outdoor_rh = outdoor_rh
        self.wind_speed = wind_speed
        self.solar_radiation = solar_radiation
        self.clo = clo
        self.met = met
        self.v = v
        self.fixed_wall_list = fixed_wall_list
        self.th_zone_list = th_zone_list

        # Bound: lower bound is 0, upper bound is initial_position[-1]
        xl = np.zeros(self.num_decision_variables)
        xu = np.ones(self.num_decision_variables) * np.max(self.position)

        # Constraints 1-6
        num_cons_1 = self.num_decision_variables - 1
        num_cons_2 = sum(temp >= 0 for temp in self.fixed_wall_list)
        num_cons_3 = 1
        num_cons_4 = self.num_decision_variables - 1
        num_cons_5 = 1
        num_cons_6 = self.num_decision_variables - 1

        super().__init__(n_var=self.num_decision_variables,
                         n_obj=2,
                         n_constr=num_cons_1 + num_cons_2 + num_cons_3 + num_cons_4 + num_cons_5 + num_cons_6,
                         xl=xl,
                         xu=xu)


    def _evaluate(self, X, out, *args, **kwargs):
        """
        X: decision variables
        """
        fixed_wall_list = self.fixed_wall_list

        room_types = self.room_types
        required_num_meeting_room = self.required_num_meeting_room
        space_width = self.space_width
        room_area_min = self.room_area_min

        initial_occ_list = self.initial_occ_list
        total_occ = self.total_occ
        occ_min_area = self.occ_min_area

        th_zone_list = self.th_zone_list

        predictor = self.predictor
        outdoor_t = self.outdoor_t
        outdoor_rh = self.outdoor_rh
        wind_speed = self.wind_speed
        solar_radiation = self.solar_radiation

        v = self.v
        met = self.met
        clo = self.clo


        # f1_cost and f2_pmv
        result_f1 = []
        result_f2 = []

        # occ_record, area_occ_record
        occ_record = np.array([])
        area_occ_record = np.zeros((X.shape[0], X.shape[1] - 1))
        room_area_list_record = np.zeros((X.shape[0], X.shape[1] - 1))

        # Step 0: Reset the fixed wall values
        fixed_indices = np.where(fixed_wall_list >= 0)[0]
        X[:, fixed_indices] = fixed_wall_list[fixed_indices]


        for i, single_solution in enumerate(X):

          # Step 1: Update room_list information: room_area_list, room_occ_list, room_t_list, room_rh_list
          room_area_list = np.diff(single_solution) * space_width
          room_area_list[room_area_list < 0] = 0
          total_room_area = np.round(np.sum(room_area_list), 1)

          num_rooms = np.sum(room_area_list > room_area_min)
          total_num_rooms = np.sum(room_area_list > -1)


          ## Update number of occupants by idea "Larger room has a higher probability to be allocated occupants"
          ## Allocate occupants based on the room_area_weights, and only to the offices
          room_occ_list = self.occ_allocate(initial_occ_list, occ_min_area, room_area_list, room_types)

          ##print(i, ' original_room_area:', np.round(room_area_list, 1),
          ##      'total area:', total_room_area,
          ##      'total rooms:', total_num_rooms,
          ##      'valid rooms:', num_rooms,
          ##      'total_new_occ:', sum(room_occ_list))

          ## Update occ_record for constraint 3: constraints_total_occ
          occ_record = np.append(occ_record, abs(sum(room_occ_list) - total_occ))

          ## Update area_occ_record for constraint 4: constraints_occ_area
          area_occ_record[i,:] = room_occ_list * occ_min_area - room_area_list

          ## Update room_area_list_record for constraint 6: constraints_room_area
          room_area_list_record[i,:] = room_area_list - room_area_min

          ## Update room t & rh by target th_zone
          room_t_list, room_rh_list = self.update_room_t_and_rh(single_solution, th_zone_list)

          result_f1_single_solution = 0 # save f1 result of one room
          result_f2_single_solution = 0 # save f2 result of one room

          # Step 2: Evaluate f1 and f2
          features_batch_f1 = []
          features_batch_f2 = []
          for room_area, room_t, room_rh, room_occ in zip(room_area_list, room_t_list, room_rh_list, room_occ_list):
            features_batch_f1.append([room_occ, room_t, room_rh, outdoor_t, outdoor_rh, wind_speed, solar_radiation])
            features_batch_f2.append([room_t, room_t, v, room_rh, met, clo])

          # f1
          objective_f1 = f1_cost(predictor=predictor)
          result_f1_single_solution = objective_f1.calculate_cost(features_batch_f1, room_area_list)

          # f2
          objective_f2 = f2_pmv()
          result_f2_single_solution = objective_f2.calculate_pmv(features_batch_f2, room_occ_list)

          result_f1.append(result_f1_single_solution)
          result_f2.append(result_f2_single_solution)


        # Step 3: Output objective values
        out["F"] = np.column_stack([result_f1, result_f2])

        # Step 4: Constraints
        ## Constraints 1: X[i] >= X[i-1]
        constraints_X = np.diff(X)
        count_cons1 = np.sum(constraints_X < 0)
        ##print(f"\nconstraint 1 (wall_position) violation: {count_cons1} solutions")

        ## Constraints 2: fixed wall, X[:, i] - fixed_wall_list[i] = 0
        constraints_fixed_walls = np.column_stack([X[:, i] - self.fixed_wall_list[i] for i in range(len(self.fixed_wall_list)) if self.fixed_wall_list[i] >= 0])
        count_cons2 = np.sum(constraints_fixed_walls > 0)
        ##print(f"constraint 2 (fixed_wall) violation: {count_cons2} solutions")

        ## Constraints 3: fixed total occ
        constraints_total_occ = occ_record.reshape(-1, 1)
        ##print(f"constraint 3 (total_occ) violation: {int(sum(occ_record))} solutions")

        ## Constraints 4: occ * occ_min_area <= room_area
        constraints_occ_area = area_occ_record
        count_4 = np.sum(constraints_occ_area > 0)
        ##print(f"constraint 4 (occ_area) violation: {count_4} solutions")

        ## Constraints 5: num_meeting_room >= required number of meeting rooms
        num_meeting_room = np.count_nonzero(room_types == 'meeting_room')
        constraints_num_meeting_room = np.full((X.shape[0], 1), required_num_meeting_room - num_meeting_room)
        count_cons5 = np.sum(constraints_num_meeting_room < 0)
        ##print(f"constraint 5 (num_meeting_room) violation: {count_cons5} solutions")

        ## Constraints 6: room_area >= room_area_min
        constraints_room_area_list = room_area_list_record
        count_cons6 = np.sum(constraints_room_area_list < 0)
        ##print(f"constraint 6 (room_area_min) violation: {count_cons6} solutions\n")
        ##print(f"total constraints violation: {count_cons1 + count_cons2 + int(sum(occ_record)) + count_4 + count_cons5 + count_cons6}\n")


        out["G"] = np.column_stack([-constraints_X,
                                    constraints_fixed_walls,
                                    constraints_total_occ,
                                    constraints_occ_area,
                                    constraints_num_meeting_room,
                                    -constraints_room_area_list])



    # !!! Self-defined function for updating room_t and room_rh !!!
    def update_room_t_and_rh(self, single_solution, th_zone_list):
        zone_start = th_zone_list[:, 0]
        zone_end = th_zone_list[:, 1]
        zone_t = th_zone_list[:, 2]
        zone_rh = th_zone_list[:, 3]

        num_intervals = len(single_solution) - 1
        room_t_list = np.zeros(num_intervals)
        room_rh_list = np.zeros(num_intervals)

        left_boundaries = single_solution[:-1]
        right_boundaries = single_solution[1:]
        room_areas = right_boundaries - left_boundaries

        # Handle penalties for invalid areas
        penalties_mask = room_areas <= 0
        room_t_list[penalties_mask] = 32.0
        room_rh_list[penalties_mask] = 100.0

        # Mask for valid areas
        valid_mask = room_areas > 0
        valid_left_boundaries = left_boundaries[valid_mask]
        valid_right_boundaries = right_boundaries[valid_mask]
        valid_room_areas = room_areas[valid_mask]

        # Find overlapping areas in vectorized form
        overlap_start = np.maximum(valid_left_boundaries[:, np.newaxis], zone_start)
        overlap_end = np.minimum(valid_right_boundaries[:, np.newaxis], zone_end)
        overlap_length = np.clip(overlap_end - overlap_start, 0, None)

        # Calculate the overlap ratios
        overlap_ratios = overlap_length / valid_room_areas[:, np.newaxis]

        # Sum temperature and humidity weighted by overlap ratios
        room_t_list[valid_mask] = np.sum(overlap_ratios * zone_t, axis=1)
        room_rh_list[valid_mask] = np.sum(overlap_ratios * zone_rh, axis=1)

        return room_t_list, room_rh_list


    def occ_allocate(self, initial_occ_list, occ_min_area, room_area_list, room_types):
        office_area_list = room_area_list * (room_types=='office')
        occ_remain = 0
        new_occ_list = initial_occ_list.copy()

        # Find remain occ should be further allocated
        max_possible_occ = np.floor(office_area_list / occ_min_area).astype(int)
        excess_occupancy = np.maximum(initial_occ_list - max_possible_occ, 0)
        occ_remain = np.sum(excess_occupancy)
        new_occ_list = initial_occ_list - excess_occupancy

        # Occupancy allocation from first room to last room
        for j, single_room_area in enumerate(office_area_list):
          max_occ_addable = (single_room_area // occ_min_area) - initial_occ_list[j]

          if max_occ_addable >= 1 and occ_remain > 0:
            occ_added = min(max_occ_addable, occ_remain)
            new_occ_list[j] += occ_added
            occ_remain -= occ_added

          if occ_remain == 0:
            break

        return new_occ_list



### **Class: CustomMutation with movable wall**

In [6]:
class custom_mutation(Mutation):
#class custom_mutation:
    def __init__(self, prob, eta, fixed_wall_list=None):
        super().__init__()
        self.prob = prob
        self.eta = eta
        self.fixed_wall_list = fixed_wall_list

    def _do(self, problem, X, **kwargs):
        # Step 1: Select the movable indices from fixed_wall_list
        movable_indices = np.where(self.fixed_wall_list < 0)[0]

        # Step 2: Generate a random mutation mask matrix of the same shape as X, and mutate where the probability of mutation is lower than self.prob
        mutation_mask = np.random.random(X[:, movable_indices].shape) < self.prob

        # Step 3: Gaussian noise
        random_num = np.random.normal(0, self.eta, size=mutation_mask.shape)
        X[:, movable_indices] += mutation_mask * random_num
        #X[:, movable_indices] += mutation_mask * np.random.normal(0, self.eta, size=mutation_mask.shape) * scale_factor #scale_factor=1 # The maximum a wall can move in a mutation is +/- 1 metre

        # Step 4: Check the bound
        X = np.clip(X, problem.xl, problem.xu)

        # Step 5: Sort the solution to prevent room area constraints violation
        X = np.sort(X, axis=1)

        # Step 6: Reset the fixed wall values
        fixed_indices = np.where(fixed_wall_list >= 0)[0]
        X[:, fixed_indices] = fixed_wall_list[fixed_indices]
        return X

### **Class: FromArraySampling**

In [7]:
class FromArraySampling(Sampling):
    def __init__(self, initial_population):
        super().__init__()
        self.initial_population = np.array(initial_population)

    def _do(self, problem, n_samples, **kwargs):
        n_initial = self.initial_population.shape[0]
        if n_initial < n_samples:
            extra = n_samples - n_initial
            random_samples = np.random.uniform(problem.xl, problem.xu, (extra, problem.n_var))
            return np.vstack([self.initial_population, random_samples])
        else:
            return self.initial_population[:n_samples]

### **Class: HVMonitor**

In [8]:
class HVMonitor:
    def __init__(self, ref_point):
        self.hv = HV(ref_point=ref_point)
        self.values = []

    def __call__(self, algorithm):
        F = algorithm.pop.get("F")
        hv_value = self.hv(F)
        self.values.append(hv_value)
        print(f"Hypervolume: {hv_value:.3e} \n")

### **Main**

In [9]:
if __name__ == "__main__":

    # Input: Default parameters
    outdoor_t = 27.5
    outdoor_rh = 85
    wind_speed = 1
    solar_radiation = 650
    clo_default = 0.5
    met_default = 1.0
    v_default = 0.1

    # Input: Initial parameters
    indoor_t = 25
    indoor_rh = 70

    # Input: Load the focasting model (autogluon)
    autogluon_predictor = TabularPredictor.load("/content/drive/MyDrive/PhD 2024/model_AutoGluon/")

    # Input: wall related information (number of walls, initial wall position)
    num_decision_variables = 7

    ## wall_position, initial solutions limit the position of first and last walls (structural walls)
    position_first_wall = 0
    position_last_wall = 60.0
    initial_solution = np.array([0.0,10.0,20.0,30.0,40.0,50.0, 60.0])

    ## fixed_wall_list, >=0 (fixed wall, positions); -1 (movable walls)
    fixed_wall_list = np.array([position_first_wall, -1 ,-1 ,-1, -1, -1, position_last_wall])


    # Input: room related imformation
    room_types_list = np.array(['office', 'office', 'office','office', 'office', 'meeting_room'])
    required_num_meeting_room = 1
    space_width = 2
    room_length_min = 3
    room_area_min = room_length_min * space_width
    print('required_num_meeting_room: ', required_num_meeting_room)

    # Input: occ related information (occ_list, occ_min_area)
    ## occ_position_list: occ number, left boundary and right boundary that the person occupies in one room
    initial_occ_list = np.array([3, 3, 3, 2, 2, 0])
    total_occ = sum(initial_occ_list)
    occ_min_area = 6


    # Input: TH zones, setting and updating room_t and room_rh
    th_zone_1 = np.array([0,10,22.5,85])
    th_zone_2 = np.array([10,20,23,82])
    th_zone_3 = np.array([20,30,23.5,80])
    th_zone_4 = np.array([30,40,22.9,86])
    th_zone_5 = np.array([40,50,22.2,89])
    th_zone_6 = np.array([50,60,22, 85])
    th_zone_list = np.array([th_zone_1, th_zone_2, th_zone_3, th_zone_4, th_zone_5, th_zone_6])
    print('th_zone_list\n', th_zone_list)


required_num_meeting_room:  1
th_zone_list
 [[ 0.  10.  22.5 85. ]
 [10.  20.  23.  82. ]
 [20.  30.  23.5 80. ]
 [30.  40.  22.9 86. ]
 [40.  50.  22.2 89. ]
 [50.  60.  22.  85. ]]


### **Experiment results**

In [12]:
###################### Hyper-parameters ######################
# Opt: Generation, population size, mutation, crossover
g = 500
pop_size = 30
mutation = custom_mutation(prob=0.7, eta=5, fixed_wall_list=fixed_wall_list)
crossover = SBX(prob=0.8)


# Opt: Define MOO problem
solutions = np.tile(initial_solution, (pop_size, 1))
solutions[:, 0] = position_first_wall
solutions[:, -1] = position_last_wall
problem = flexible_wall_problem(num_decision_variables=num_decision_variables, initial_position=solutions, fixed_wall_list = fixed_wall_list,
                                  room_types=room_types_list, required_num_meeting_room = required_num_meeting_room, space_width = space_width, room_area_min = room_area_min,
                                  initial_occ_list = initial_occ_list, total_occ = total_occ, occ_min_area = occ_min_area,
                                  th_zone_list = th_zone_list,
                                  predictor=autogluon_predictor,
                                  indoor_t=indoor_t, indoor_rh=indoor_rh, outdoor_t=outdoor_t, outdoor_rh=outdoor_rh, wind_speed=wind_speed,solar_radiation=solar_radiation,
                                  clo=clo_default, met=met_default, v=v_default)


# Opt: Initial solution
original_solution = initial_solution.reshape(1, -1)
initial_pop = Population.new("X", original_solution)
result = problem.evaluate(initial_pop.get("X"))
initial_f1 = result[0][0][0]
initial_f2 = result[0][0][1]
print('\nInitial objective values')
print(f"f1: {initial_f1:.3f}, f2: {initial_f2:.3f} \n")


###################### Algorithms ######################

# Opt: NSGA2, generations, initialize population
algorithm = NSGA2(pop_size=pop_size,
                      mutation=mutation,
                      crossover=crossover,
                      eliminate_duplicates=True,
                      sampling=FromArraySampling(solutions))
termination = get_termination("n_gen", g)

"""
# Opt: NSGA3, generations
ref_dirs_NSGA3 = get_reference_directions("das-dennis", 2, n_partitions=12)
algorithm = NSGA3(pop_size=pop_size,
                        mutation=mutation,
                        crossover=crossover,
                        ref_dirs=ref_dirs_NSGA3,
                        eliminate_duplicates=True,
                        sampling=FromArraySampling(solutions))
termination = get_termination("n_gen", g)
"""
"""
# Opt: SMS-EMOA, generations
algorithm = SMSEMOA(pop_size=pop_size,
                            mutation=mutation,
                            crossover=crossover,
                            eliminate_duplicates=True,
                            sampling=FromArraySampling(solutions))
termination = get_termination("n_gen", g)
"""

"""
# Opt: R-NSGA2, generations
ref_points = np.array([[0, 0]])
algorithm = RNSGA2(pop_size=pop_size,
                          mutation=mutation,
                          crossover=crossover,
                          ref_points=ref_points,
                          eliminate_duplicates=True,
                          sampling=FromArraySampling(solutions))
termination = get_termination("n_gen", g)
"""

###################### Main optimization ######################
for random_seed in range(1):
  ref_point = np.array([0.5, 1.2])
  start_time = time.time()
  res = minimize(problem,
                algorithm,
                termination,
                seed=random_seed,
                save_history=True,
                verbose=True,
                callback=HVMonitor(ref_point=ref_point))
  end_time = time.time()
  print(f"Running time: {end_time - start_time:.3f} s")

  # Opt: final optimal solutions and objective function values
  print("Pareto-optimal solutions (X):")
  print(res.X)
  print('\nOptimal objective values')
  for i in range(len(res.F)):
    f1_reduce_percent = ((initial_f1-res.F[i][0])/initial_f1) * 100
    f2_reduce_percent = ((initial_f2-res.F[i][1])/initial_f2) * 100
    print(f"solution_{i+1} f1: {res.F[i][0]:.3f} ({f1_reduce_percent:.3f}%), f2: {res.F[i][1]:.3f} ({f2_reduce_percent:.3f}%)\n")



Initial objective values
f1: 0.326, f2: 0.938 

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        1 |      1 |  0.000000E+00 |  0.000000E+00 |             - |             -
Hypervolume: 4.572e-02 

     2 |       31 |      3 |  0.000000E+00 |  2.0610100326 |  1.6338547627 |         ideal
Hypervolume: 6.340e-02 



KeyboardInterrupt: 

In [ ]:
# Opt: plot
plot = Scatter()
plot.add(res.F)
plot.show()

plotter = plot_2D_display(num_decision_variables, initial_solution, room_types_list, space_width)
plotter.plot()

plotter = plot_2D_display(num_decision_variables, res.X[0], room_types_list, space_width)
plotter.plot()

In [ ]:
# Hypervolume
ref_point = np.array([1.0, 2.0])
hv = HV(ref_point=ref_point)
hypervolume = hv(res.F)
print("Hypervolume:", hypervolume)

### **Test**
